In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Browser Function Calling with the Vertex AI Gemini API & Python SDK

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/intro_function_calling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/intro_function_calling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/intro_function_calling.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


In [2]:
#upgrade the sdk for Gemini
# !pip3 install --upgrade --user google-cloud-aiplatform

In [3]:
# !pip install playwright > /dev/null
# !pip install  lxml

# If this is your first time using playwright, you'll have to install a browser executable.
# Running `playwright install` by default installs a chromium browser executable.
# !playwright install
# Also run this:
# !playwright install dependencies

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, it is recommended to restart the runtime. Run the following cell to restart the current kernel.

The restart process might take a minute or so.

In [1]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

: 

#### We will be utilizing pre-built Langchain tools for the Google Search

____________

## Google Search
[Langchain Guide](https://python.langchain.com/docs/integrations/tools/google_search)

This notebook goes over how to use the google search component.

First, you need to set up the proper API keys and environment variables. To set it up, create the GOOGLE_API_KEY in the Google Cloud credential console (https://console.cloud.google.com/apis/credentials) and a GOOGLE_CSE_ID using the Programmable Search Engine (https://programmablesearchengine.google.com/controlpanel/create). Next, it is good to follow the instructions found here.

Then we will need to set some environment variables.

```python
import os

os.environ["GOOGLE_CSE_ID"] = ""
os.environ["GOOGLE_API_KEY"] = ""
```

In [1]:
#using a local json key - careful don't commit that file!
import json
import os

f = open('config.json')
config = json.load(f)
f.close()


os.environ["GOOGLE_CSE_ID"] = config['SEARCH_ID']
os.environ["GOOGLE_API_KEY"] = config['CSE_KEY']

In [2]:

import requests
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

from langchain.tools import Tool as LangchainTool
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()


2023-12-15 23:28:13.583467: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-15 23:28:14.920108: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-12-15 23:28:14.920259: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or di

#### Create the Langchain tool for search

In [3]:
tool = LangchainTool(
    name="Google Search",
    description="Search Google for recent results.",
    func=search.run,
)

In [4]:
tool.run("Obama's first name?")

"1 Child's First Name. (Type or print). BARACK. CERTIFICATE OF LIVE BIRTH lb ... OBAMA, II. Day. 4. 6b. Island. Year. 5b. Hour. 1961 7:24 P.M.. Oahu. 6d. Is Place\xa0... His last name, Obama, was derived from his Luo descent. Obama's parents met in ... First 100 days. Main article: First 100 days of Barack Obama's presidency. Apr 12, 2017 ... President Barack Obama's full name is Barack Hussein Obama. Does it mean that he is a Muslim? 199,232 Views. First Lady Michelle LaVaughn Robinson Obama is a lawyer, writer, and the wife of the 44th President, Barack Obama. She is the first African-American First\xa0... Apr 2, 2018 ... BARACK : Barkat and Mubarak both are derived from it in Hindi and Urdu. Roughly meaning blessing, abundance etc. Husen or Hussein from which\xa0... Barack Hussein Obama II was born August 4, 1961, in Honolulu, Hawaii, to parents Barack H. Obama, Sr., and Stanley Ann Dunham. His parents divorced when he\xa0... Jan 19, 2017 ... Hopeful parents named their sons for the

#### Get the tool parameters
Looking at the source code to find the parameters for the `.run` method for the tool


```python
def run(self, query: str) -> str:
        """Run query through GoogleSearch and parse result."""
        ...
```

In [5]:
search_func = FunctionDeclaration(
    name="google_search",
    description="Perform a Google Search",
    parameters={
    "type": "object",
    "properties": {
        "query": {
            "type": "string",
            "description": "Google Search Query"
        }
    }
},
)


You can then define a tool for the LLM to call that includes the `search_func`:

In [6]:
search_tool = Tool(
    function_declarations=[search_func],
)

### Use the Gemini Pro model

The Gemini Pro (`gemini-pro`) model is designed to handle natural language tasks, multiturn text and code chat, and code generation.

In [7]:
model = GenerativeModel("gemini-pro")

In [8]:
prompt = "What is the top news on Panera Bread?"

response = model.generate_content(
    prompt,
    generation_config={"temperature": 0},
    tools=[search_tool]
)
response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "google_search"
        args {
          fields {
            key: "query"
            value {
              string_value: "Top news on Panera Bread"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 10
  total_token_count: 10
}

##### Quick inspection of one function call

In [9]:
response.candidates[0].content.parts[0].function_call

name: "google_search"
args {
  fields {
    key: "query"
    value {
      string_value: "Top news on Panera Bread"
    }
  }
}

In [10]:
from google.cloud.aiplatform_v1beta1.types.tool import FunctionCall
from vertexai.generative_models._generative_models import GenerationResponse
from typing import List

def get_parts_from_response(response: GenerationResponse) -> List[FunctionCall]:
    """
    Get the parts from a generation response.

    Args:
        response: The generation response.

    Returns:
        A list of parts.
    """
    return(response.candidates[0].content)

#### We will now do a mult-turn chat
Example can be found here on [function calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling#function-calling-one-and-a-half-turn-curl-sample)

In [11]:
model = GenerativeModel("gemini-pro", 
                        generation_config={"temperature": 0},
                        tools=[search_tool])
chat = model.start_chat()

In [12]:
from pprint import pprint

prompt = "How did deepmind recently solve an unsolvable math problem (december 2023)"
# while_loop = True

# while while_loop:
response = chat.send_message(prompt)
# pprint(response)
function_calls = get_parts_from_response(response)

function_calls = function_calls.parts
function_calls

[function_call {
   name: "google_search"
   args {
     fields {
       key: "query"
       value {
         string_value: "Deepmind solved unsolvable math problem December 2023"
       }
     }
   }
 }]

In [13]:
single_function_args = function_calls[0].function_call.args
tool_res = tool.run( {k: single_function_args[k] for k in single_function_args})
print(tool_res)

1 day ago ... 14 December 2023. DeepMind AI outdoes human mathematicians on ... Related Articles. DeepMind AI invents faster algorithms to solve tough maths ... Google DeepMind used a large language model to solve ... December 14, 2023. scraps of old algebra textbook piled with an old photo of a woman surrounded by ... Posted by BeauHD on Thursday December 14, 2023 @10:30PM from the what-will-they-think-of-next dept. An anonymous reader quotes a report from MIT Technology ... 10 hours ago ... -> Google DeepMind's FunSearch model cracks the unsolved cap set problem in pure mathematics, the first time an LLM has solved a long-standing ... Dec 7, 2021 ... The AI comes just a few months after DeepMind's previous triumph in solving a 50-year-old challenge in biology. ... December 9, 2023. Load more. UPDATED 18:50 EST / DECEMBER 14 2023. AI. Google's DeepMind creates generative AI model with fact checker to crack unsolvable math problem. by Mike Wheatley. 21 hours ago ... To build “FunSearch

In [14]:
model_response = chat.send_message(
  Part.from_function_response(
      name="google_search",
      response={
          "content": {"google_search_results": tool_res},
      }
  ),
  tools=[search_tool]
)
print("model_response\n", model_response)

model_response
 candidates {
  content {
    role: "model"
    parts {
      text: " DeepMind, a leading artificial intelligence research company, recently made a breakthrough in solving a previously unsolvable math problem. The problem, known as the \"Cap Set Problem,\" had stumped mathematicians for decades. DeepMind\'s solution was published in the journal Nature in December 2023.\n\nThe Cap Set Problem involves finding the maximum number of non-overlapping sets that can be formed from a given set of elements. For example, if you have the set {1, 2, 3}, you can form the following sets:\n\n* {1}\n* {2}\n* {3}\n* {1, 2}\n* {1, 3}\n* {2, 3}\n\nThe maximum number of non-overlapping sets that can be formed from this set is 6.\n\nDeepMind\'s solution to the Cap Set Problem is based on a new algorithm called the \"FunSearch\" algorithm. The FunSearch algorithm uses a technique called \"reinforcement learning\" to search for solutions to the problem. Reinforcement learning is a type of mach

In [15]:
chat.send_message(prompt)

candidates {
  content {
    role: "model"
    parts {
      text: "DeepMind\'s solution to the Cap Set Problem involved developing a novel algorithm called \"FunSearch.\" FunSearch leverages reinforcement learning, a technique that enables algorithms to learn from their mistakes and refine their performance over time.\n\nThe FunSearch algorithm operates by iteratively generating potential solutions to the Cap Set Problem. It evaluates each solution based on its effectiveness in maximizing the number of non-overlapping sets. Solutions that yield better outcomes are reinforced, while those that perform poorly are gradually discarded.\n\nThrough this iterative process, FunSearch progressively enhances its ability to identify high-quality solutions. It explores various combinations and arrangements of elements within the given set, seeking to uncover the optimal configuration that maximizes the number of non-overlapping sets.\n\nDeepMind\'s application of FunSearch to the Cap Set Problem 

In [16]:
followup_prompt = "what are other related google search topics on this I could research? Use the google search tool"

In [17]:
response = chat.send_message(followup_prompt)
function_calls = get_parts_from_response(response)

function_calls = function_calls.parts
single_function_args = function_calls[0].function_call.args
tool_res = tool.run( {k: single_function_args[k] for k in single_function_args})
print(tool_res)

1 day ago ... Google DeepMind used a large language model to solve an unsolvable math problem. They had to throw away most of what it produced but there ... 24 hours ago ... Built on top of DeepMind's game-playing AI AlphaZero, both solved math problems by treating them as if they were puzzles in Go or chess. The ... 1 day ago ... ... to solve maths problems with known solutions, says Kohli. ... Related Articles. DeepMind AI invents faster algorithms to solve tough maths puzzles. Feb 23, 2007 ... ... problems, and other maths-related topics. ... Pingback: Google DeepMind used a large language model to solve an unsolvable math problem – Maverick ... 24 hours ago ... When tasked with mathematical problems, FunSearch writes computer code that can find the solution, rather than trying to tackle it directly. 1 day ago ... Haven't mathematicians been using complex computer modeling to help solve unsolved math problems since computers have existed? And havent ... 7 hours ago ... Google's AI d

In [18]:
model_response = chat.send_message(
  Part.from_function_response(
      name="google_search",
      response={
          "content": {"google_search_results": tool_res},
      }
  ),
  tools=[search_tool]
)
print("model_response\n", model_response)

model_response
 candidates {
  content {
    role: "model"
    parts {
      text: " Here are some related Google search topics on DeepMind solving an unsolvable math problem in December 2023 that you could research:\n\n* **DeepMind\'s FunSearch algorithm:** Learn more about the specific algorithm that DeepMind developed to solve the Cap Set Problem. Explore how FunSearch leverages reinforcement learning and its advantages over traditional mathematical approaches.\n* **Applications of FunSearch:** Investigate potential applications of the FunSearch algorithm beyond the Cap Set Problem. Consider other mathematical domains or real-world scenarios where FunSearch could be applied to solve complex optimization problems.\n* **Limitations of FunSearch:** Identify any limitations or challenges associated with using FunSearch. Explore scenarios where the algorithm may not be suitable or effective, and consider potential areas for improvement or further research.\n* **Ethical implications:** Re

In [19]:
final_prompt="please create a nice bulleted summary of this topic"
final_response = chat.send_message(final_prompt)

pprint(final_response.candidates[0].content.parts[0].text)

("* **DeepMind's Breakthrough:**\n"
 '    * Solved the Cap Set Problem, a previously unsolvable math problem, in '
 'December 2023.\n'
 '    * Developed a novel algorithm called FunSearch based on reinforcement '
 'learning.\n'
 '* **FunSearch Algorithm:**\n'
 '    * Iteratively generates and evaluates potential solutions to the Cap Set '
 'Problem.\n'
 '    * Reinforces high-quality solutions and discards poor ones, improving '
 'performance over time.\n'
 '    * Treats the problem as an optimization task, seeking to maximize the '
 'number of non-overlapping sets.\n'
 '* **Significance:**\n'
 '    * Demonstrates the power of AI to tackle complex mathematical problems.\n'
 '    * Opens up new avenues for solving problems that have eluded traditional '
 'approaches.\n'
 '    * Highlights the potential of reinforcement learning in mathematical '
 'research.\n'
 '* **Related Topics for Further Research:**\n'
 '    * Applications of FunSearch beyond the Cap Set Problem.\n'
 '    * Limitat